# Q1: SQL with pandas
* <a href="https://pandas.pydata.org/docs/getting_started/comparison/comparison_with_sql.html#comparison-with-sql">Pandas Comparison with SQL</a>
* <a href="https://pandas.pydata.org/docs/user_guide/10min.html#min">10 minutes to pandas-Basic functions of Pandas</a>

# Q2: Modeling

## Regression

## Classification